# Running Parquez pipeline

### create the mlrun project 

In [1]:
!/User/align_mlrun.sh

Both server & client are aligned (0.6.1).


In [2]:
from os import path, getenv
from mlrun import new_project, mlconf

#project_name = '-'.join(filter(None, ['getting-started-iris', getenv('V3IO_USERNAME', None)]))
project_name = "parquez"
project_path = path.abspath('./')
project = new_project(project_name, project_path)
project.save()
print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/parquez
Project name: parquez


In [3]:
out = mlconf.artifact_path or path.abspath('./data')
# {{run.uid}} will be substituted with the run id, so output will be written to different directoried per run
artifact_path = path.join(out, '{{run.uid}}')
%env PYTHONPATH=./

env: PYTHONPATH=./


### set the project functions

In [4]:
image_name =  'aviaigz/parquez:0.6.1'

project.set_function("functions/clean_parquez.py", 'clean', kind='job', image=image_name)
project.set_function("functions/validate_input.py", 'validate', kind='job', image=image_name)
project.set_function("functions/get_table_schema.py", 'get_schema', kind='job', image=image_name)
project.set_function("functions/create_parquet_table.py", 'create_parquet', kind='job', image=image_name)
project.set_function("functions/create_unified_view.py", 'create_unified_view', kind='job', image=image_name)
project.set_function("functions/parquet_add_partition.py", 'parquet_add_partition', kind='job', image=image_name)
project.set_function("functions/run_scheduler.py", 'run_scheduler', kind='job', image=image_name)

### deploy kv-to-parquet function 

In [5]:
from mlrun import new_function
#should be converted to an image (spark operator)
spark_job = new_function(kind='spark', command='/User/parquez/functions/kv_to_parquet.py', name='kv-to-parquet') # /User isn't supported at this stage
project.set_function(spark_job)
project.func('kv-to-parquet').with_driver_limits(cpu="4000")
project.func('kv-to-parquet').with_driver_requests(cpu=2, mem="2g") # gpu_type & gpus=<number_of_gpus> are supported too
project.func('kv-to-parquet').with_executor_limits(cpu="4000")
project.func('kv-to-parquet').with_executor_requests(cpu=2, mem="2g")
project.func('kv-to-parquet').with_igz_spark() # Adds fuse, daemon & iguazio's jars support
project.func('kv-to-parquet').deploy()# Rebuilds the image with MLRun - This is needed in order to support artifact logging etc. This step is too long (~3 minutes)

> 2021-04-12 12:08:07,136 [info] running build to add mlrun package, set with_mlrun=False to skip if its already in the image
> 2021-04-12 12:08:07,137 [info] starting remote build, image: .mlrun/func-parquez-kv-to-parquet-latest
E0412 12:08:49.974495       1 aws_credentials.go:77] while getting AWS credentials NoCredentialProviders: no valid providers in chain. Deprecated.
	For verbose messaging see aws.Config.CredentialsChainVerboseErrors
INFO[0040] Retrieving image manifest datanode-registry.iguazio-platform.app.dev39.lab.iguazeng.com:80/iguazio/spark-app:3.0_b51_20210308021033 
INFO[0040] Retrieving image manifest datanode-registry.iguazio-platform.app.dev39.lab.iguazeng.com:80/iguazio/spark-app:3.0_b51_20210308021033 
INFO[0040] Built cross stage deps: map[]                
INFO[0040] Retrieving image manifest datanode-registry.iguazio-platform.app.dev39.lab.iguazeng.com:80/iguazio/spark-app:3.0_b51_20210308021033 
INFO[0040] Retrieving image manifest datanode-registry.iguazio-pla

True

In [6]:
from mlrun import mount_v3io

# create-kv-view
project.set_function("functions/create_kv_view.py", 'create-kv-view', kind='job', image=image_name)
project.func('create-kv-view').apply(mount_v3io())
project.func('create-kv-view').set_env('PYTHONPATH', project_path)
project.func('create-kv-view').deploy(with_mlrun=False)

True

In [7]:
# parquet-add-partition
project.set_function("functions/parquet_add_partition.py", 'parquet-add-partition', kind='job', image=image_name)
project.func('parquet-add-partition').apply(mount_v3io())
project.func('parquet-add-partition').set_env('PYTHONPATH', project_path)
project.func('parquet-add-partition').deploy(with_mlrun=False)

True

In [16]:
#delete-kv-partition
project.set_function("functions/delete_kv_partition.py", 'delete-kv-partition', kind='job', image=image_name)
project.func('delete-kv-partition').apply(mount_v3io())
project.func('delete-kv-partition').set_env('PYTHONPATH', project_path)
project.func('delete-kv-partition').deploy(with_mlrun=False)

True

In [17]:
#delete-kv-partition
project.set_function("functions/delete_historical_retention.py", 'delete-historical-retention', kind='job', image=image_name)
project.func('delete-historical-retention').apply(mount_v3io())
project.func('delete-historical-retention').set_env('PYTHONPATH', project_path)
project.func('delete-historical-retention').deploy(with_mlrun=False)

True

In [18]:
#parquetinizer
project.set_function("functions/parquetinizer.py", 'parquetinizer', kind='job', image=image_name)
project.func('parquetinizer').apply(mount_v3io())
project.func('parquetinizer').set_env('PYTHONPATH', project_path)
project.func('parquetinizer').deploy(with_mlrun=False)

True

<a id="gs-step-create-n-run-ml-pipeline"></a>
## Create and Run a Fully Automated ML Pipeline

You're now ready to create a full ML pipeline.
This is done by using [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/), which is integrated into the Iguazio Data Science Platform.
Kubeflow Pipelines is an open-source framework for building and deploying portable, scalable machine-learning workflows based on Docker containers.
MLRun leverages this framework to take your existing code and deploy it as steps in the pipeline.

In [10]:
%%writefile {path.join(project_path, 'workflow.py')}

from kfp import dsl
from mlrun import mount_v3io
from os import path
import os
from mlrun import load_project

project_path = path.abspath('./')
project = load_project(project_path)
PROJECT_NAME = project.name



V3IO_ACCESS_KEY = os.environ['V3IO_ACCESS_KEY']
V3IO_USERNAME = os.getenv('V3IO_USERNAME')


funcs = {}
project_path = path.abspath('./')
parquez_params = {'view_name':'view_name'
         ,'partition_by':'h'
         ,'partition_interval':'1h'
         ,'real_time_window':'3h'
         ,'historical_retention':'24h'
         ,'real_time_table_name':'faker'
         ,'config_path':'/User/parquez/config/parquez.ini'
         ,'user_name':V3IO_USERNAME
         ,'access_key':V3IO_ACCESS_KEY          
         ,'project_path': PROJECT_NAME
         }


# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    project_path = path.abspath('./')
    for f in functions.values():
        f.apply(mount_v3io())
        f.set_env('PYTHONPATH', project_path)
        f.spec.artifact_path = 'User/artifacts'
        f.spec.service_account='mlrun-api'
        
        
# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(
    name = "parquez-pipeline",
    description = "parquez description"
)
def kfpipeline():
    
    # clean the tables
    clean = funcs['clean'].as_step(
        name="clean",
        params=parquez_params,
        outputs=['clean']
    )
    
    # Ingest the data set
    validate = funcs['validate'].as_step(
        name="validate",
        params=parquez_params,
        inputs={'table': clean.outputs},
        outputs=['validate']
    )
    
    # Analyze the dataset
    schema = funcs['get_schema'].as_step(
        name="get_schema",
        params = parquez_params,
        inputs={'table': validate.outputs},                       
        outputs=['schema']
    )
    
    parquet = funcs["create_parquet"].as_step(
        name="create_parquet",
        params=parquez_params,
        inputs={"table": schema.outputs['schema']},
        outputs=['create_parquet']
    )
    
    kv_view = funcs["create-kv-view"].as_step(
        name="create_kv_view",
        params=parquez_params,
        inputs={'table': parquet.outputs},
        outputs=['kv_view']
    )
    
    unified_view = funcs["create_unified_view"].as_step(
        name="create_unified_view",
        params=parquez_params,
        inputs={'table': kv_view.outputs},
        outputs=['unified_view']
    )
    
    unified_view = funcs["run_scheduler"].as_step(
        name="run_scheduler",
        params=parquez_params,
        inputs={'table': unified_view.outputs},
        outputs=['run_scheduler']
    )    

Overwriting /User/parquez/workflow.py


<a id="gs-register-workflow"></a>
#### Register the Workflow

Use the `set_workflow` MLRun project method to register your workflow with MLRun.
The following code sets the `name` parameter to the selected workflow name ("main") and the `code` parameter to the name of the workflow file that is found in your project directory (**workflow.py**).

In [11]:
# Register the workflow file as "main"
project.set_workflow('main', 'workflow.py')

In [12]:
project.save()

In [13]:
run_id = project.run(
    'main',
    arguments={}, 
    
    artifact_path=path.abspath(path.join('pipeline','{{workflow.uid}}'),
    
                              )
    ,dirty=True)

> 2021-04-12 12:13:07,799 [info] using in-cluster config.


> 2021-04-12 12:13:08,853 [info] Pipeline run id=fe26a23e-4a8b-4bb6-8e4c-b41bcad75b68, check UI or DB for progress


In [15]:
from mlrun import get_run_db
get_run_db().list_schedules('parquez')

SchedulesOutput(schedules=[])